In [3]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    from app_store_scraper import AppStore
    from pprint import pprint
    import pandas as pd
    import numpy as np
    import json
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install google-play-scraper
    clear_output()
    %pip install app-store-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    print("All modules required were sucessfully installed!")

Modules required are already installed!


In [ ]:
#@title App Reviews from Google Store
from google_play_scraper import app, search, Sort, reviews, reviews_all
from pprint import pprint
import pandas as pd
import numpy as np
import json
#static variables:
language = 'en'
countries = 'us'
appid = []

def searches(nameofSearch):
  result = search(
    nameofSearch,
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
  return result

def getAppId(r):
  for d in r:
    id = (d['appId'])
    # appid.append(id)
    return id

def insertId(g, cont, rating, choice):

  if choice == None:
    if rating == None:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont # defaults to 100
      # defaults to None(means all score)
    )
      return result
    else:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont, # defaults to 100
      filter_score_with=rating# defaults to None(means all score)
    )
      return result
 
  else:
    if rating == None:
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=None
      )
      return result
    else: 
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=rating # defaults to None(means all score)
      )
      return result

def googleplay(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews

def googleplay1(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews 


appfile = open("googleplay-apps.txt", "r")
#appfile = open("C:\\Users\\nhant\\Research\\apps.txt", "r")
line = appfile.readline()
choice = "Yes" # All reviews
count = None
cond = "No" # No specific reviews
rating = None
while(line):
    app = line

    googleplaydf = pd.DataFrame(np.array(googleplay(app, count, rating, choice)),columns=['review'])
    googleplaydf2 = googleplaydf.join(pd.DataFrame(googleplaydf.pop('review').tolist()))
    #display(googleplaydf2)
    googleplaydf2.to_csv('googleplay-review-' + app.strip() + '.csv')

    line = appfile.readline()


In [ ]:
#@title App Reviews from App Store
from app_store_scraper import AppStore
from pprint import pprint
import pandas as pd
import numpy as np
import json

appfile = open("applestore-apps.txt", "r")
line = appfile.readline()
count = 100000

while(line):
    app = line.strip()

    applereviews = AppStore(country = "us", app_name = app)
    applereviews.review(how_many = count)
    applereviews.reviews
    applestoredf = pd.DataFrame(np.array(applereviews.reviews),columns=['review'])
    applestoredf2 = applestoredf.join(pd.DataFrame(applestoredf.pop('review').tolist()))
    
    applestoredf2.to_csv('applestore-review-' + app + '.csv')
    print(app)

    line = appfile.readline()

In [ ]:
# metadf = pd.DataFrame(columns = ['Name', 'ReviewCount', 'AverageRating', 'AverageReviewLength', 'EarliestDate', 'LatestDate', 'AppleStoreReviews', 'GooglePlayReviews'])
# appfile = open("common-apps.txt", "r")
# line = appfile.readline()

# while(line):
#     app = line.strip()

#     applestoredf = pd.read_csv('applestore-review-' + app + '.csv')
#     googleplaydf = pd.read_csv('googleplay-review-' + app + '.csv')
#     googleplaydf = googleplaydf.rename(columns={"content": "review", "score": "rating", "at": "date"})
#     cdf = pd.concat([applestoredf, googleplaydf], ignore_index=True)

#     new_row = pd.Series({'Name': app, 'ReviewCount': cdf.shape[0], 'AverageRating': cdf.rating.mean(), 'AverageReviewLength': cdf.review.str.len().mean(), 'EarliestDate': cdf.date.min(), 'LatestDate': cdf.date.max(), 
#     'AppleStoreReviews': applestoredf.shape[0], 'GooglePlayReviews': googleplaydf.shape[0]})
#     metadf = pd.concat([metadf, new_row.to_frame().T], ignore_index=True)
#     print(app)

#     line = appfile.readline()

# print(metadf)
# metadf.to_csv('applestore-review-metadata.csv')


In [9]:
metadf = pd.DataFrame(columns = ['Name', 'ReviewCount', 'AverageRating', 'AverageReviewLength', 'EarliestDate', 'LatestDate'])
appfile = open("applestore-apps.txt", "r")
line = appfile.readline()

while(line):
    app = line.strip()

    applestoredf = pd.read_csv('./review-files/applestore-review-' + app + '.csv')
    # googleplaydf = pd.read_csv('googleplay-review-' + app + '.csv')
    # googleplaydf = googleplaydf.rename(columns={"content": "review", "score": "rating", "at": "date"})
    # cdf = pd.concat([applestoredf, googleplaydf], ignore_index=True)

    new_row = pd.Series({'Name': app, 'ReviewCount': applestoredf.shape[0], 'AverageRating': applestoredf.rating.mean(), 'AverageReviewLength': applestoredf.review.str.len().mean(), 'EarliestDate': applestoredf.date.min(), 'LatestDate': applestoredf.date.max()})
    metadf = pd.concat([metadf, new_row.to_frame().T], ignore_index=True)
    print(app)

    line = appfile.readline()

print(metadf)
metadf.to_csv('applestore-review-metadata.csv', index=False)

codeacademy
datacamp
encode
learn-python-programiz
mimo
programming-hero
programming-hub
python-x
sololearn
                     Name ReviewCount AverageRating AverageReviewLength   
0             codeacademy         473      3.171247           208.27907  \
1                datacamp         341      4.111437          215.642229   
2                  encode         184      4.396739          239.423913   
3  learn-python-programiz          86      4.511628           143.44186   
4                    mimo        2000        4.0515            240.3495   
5        programming-hero         131      4.480916          108.122137   
6         programming-hub         867      4.311419          137.641292   
7                python-x          19      4.263158          302.736842   
8               sololearn        1702      4.153349          209.386604   

          EarliestDate           LatestDate  
0  2018-08-01 01:33:12  2023-03-24 22:07:12  
1  2017-10-19 13:14:06  2023-03-14 15:45:30  
2  

In [7]:
metadf = pd.DataFrame(columns = ['Name', 'ReviewCount', 'AverageRating', 'AverageReviewLength', 'EarliestDate', 'LatestDate'])
appfile = open("googleplay-apps.txt", "r")
line = appfile.readline()

while(line):
    app = line.strip()

    # applestoredf = pd.read_csv('./review-files/applestore-review-' + app + '.csv')
    googleplaydf = pd.read_csv('./review-files/googleplay-review-' + app + '.csv')
    googleplaydf = googleplaydf.rename(columns={"content": "review", "score": "rating", "at": "date"})
    # cdf = pd.concat([applestoredf, googleplaydf], ignore_index=True)

    new_row = pd.Series({'Name': app, 'ReviewCount': googleplaydf.shape[0], 'AverageRating': googleplaydf.rating.mean(), 'AverageReviewLength': googleplaydf.review.str.len().mean(), 'EarliestDate': googleplaydf.date.min(), 'LatestDate': googleplaydf.date.max()})
    metadf = pd.concat([metadf, new_row.to_frame().T], ignore_index=True)
    print(app)

    line = appfile.readline()

print(metadf)
metadf.to_csv('googleplay-review-metadata.csv', index=False)

codeacademy
datacamp
encode
learn-python-programiz
mimo
programming-hero
programming-hub
sololearn
                     Name ReviewCount AverageRating AverageReviewLength   
0             codeacademy        3100      3.846129           70.236774  \
1                datacamp        7378      4.495798           69.904446   
2                  encode        1861      4.653949           77.064481   
3  learn-python-programiz        4728      4.656091           59.310068   
4                    mimo       86724      4.545212           59.217061   
5        programming-hero       16407       4.78308           56.691839   
6         programming-hub       48682      4.587507           59.545398   
7               sololearn      130851      4.724809           59.927417   

          EarliestDate           LatestDate  
0  2018-08-02 08:35:59  2023-03-24 00:51:56  
1  2017-10-18 11:23:14  2023-03-24 21:09:12  
2  2016-03-13 23:11:32  2023-03-19 01:47:34  
3  2021-02-19 03:24:40  2023-03-25 13:34:

In [18]:
metadf = pd.DataFrame(columns = ['Name', 'ReviewCount', 'AverageRating', 'AverageReviewLength', 'EarliestDate', 'LatestDate'])
appfile = open("googleplay-apps.txt", "r")
line = appfile.readline()

while(line):
    app = line.strip()
    if app == 'encode':
        line = appfile.readline()
        continue

    # applestoredf = pd.read_csv('./review-files/applestore-review-' + app + '.csv')
    googleplaydf = pd.read_csv('./review-files-Spanish/' + app + '_Spanish.csv')
    googleplaydf = googleplaydf.rename(columns={"content": "review", "score": "rating", "at": "date"})
    # cdf = pd.concat([applestoredf, googleplaydf], ignore_index=True)

    new_row = pd.Series({'Name': app, 'ReviewCount': googleplaydf.shape[0], 'AverageRating': googleplaydf.rating.mean(), 'AverageReviewLength': googleplaydf.review.str.len().mean(), 'EarliestDate': googleplaydf.date.min(), 'LatestDate': googleplaydf.date.max()})
    metadf = pd.concat([metadf, new_row.to_frame().T], ignore_index=True)
    print(app)

    line = appfile.readline()

print(metadf)
metadf.to_csv('googleplay-review-metadata_Spanish.csv', index=False)

codeacademy
datacamp
learn-python-programiz
mimo
programming-hero
programming-hub
sololearn
                     Name ReviewCount AverageRating AverageReviewLength   
0             codeacademy         141      4.042553           78.042553  \
1                datacamp         389      4.655527           87.120823   
2  learn-python-programiz          28      4.321429           92.464286   
3                    mimo        9253      4.380849           85.069491   
4        programming-hero         221      4.524887           97.773756   
5         programming-hub        1633      4.257195           90.685242   
6               sololearn       28876      4.751212           77.074906   

          EarliestDate           LatestDate  
0  2019-05-05 02:40:44  2023-03-21 19:53:03  
1  2018-01-27 02:54:15  2023-05-01 16:35:48  
2  2020-01-04 15:02:16  2023-04-18 23:24:33  
3  2018-06-22 22:18:37  2023-05-07 13:09:01  
4  2019-02-16 00:40:53  2023-04-10 06:35:09  
5  2014-01-12 19:12:51  2023-05

In [19]:
metadf = pd.read_csv('googleplay-review-metadata_Spanish.csv')
print(metadf.to_markdown())

|    | Name                   |   ReviewCount |   AverageRating |   AverageReviewLength | EarliestDate        | LatestDate          |   PositiveReviewProportion |   NeutralReviewProportion |   NegativeReviewProportion |
|---:|:-----------------------|--------------:|----------------:|----------------------:|:--------------------|:--------------------|---------------------------:|--------------------------:|---------------------------:|
|  0 | codeacademy            |           141 |         4.04255 |               78.0426 | 2019-05-05 02:40:44 | 2023-03-21 19:53:03 |                   0.567376 |                  0.219858 |                  0.212766  |
|  1 | datacamp               |           389 |         4.65553 |               87.1208 | 2018-01-27 02:54:15 | 2023-05-01 16:35:48 |                   0.786632 |                  0.125964 |                  0.0874036 |
|  2 | learn-python-programiz |            28 |         4.32143 |               92.4643 | 2020-01-04 15:02:16 | 2023-04-